In [1]:
import pandas as pd
import dask.dataframe as dd

from category_encoders.binary import BinaryEncoder

## Функция для чтения данных

In [5]:
def read_data(path,columns):
    data = pd.DataFrame()
    for name_column, column_type in columns.items():
        df_column = pd.DataFrame()
        for i in range(10):
            data_features_file = dd.read_parquet(
                path + f'\\part-0000{i}-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet')
            df_column = pd.concat([df_column, data_features_file[name_column].compute()], ignore_index=True)
        data[name_column] = df_column.astype(column_type)
    return data

## Популярный регион, всего регионов binary encoder

In [6]:
path = '..\\full_data\\competition_data_final_pqt'
columns = {'user_id': 'int32', 'region_name': 'category'}
data = read_data(path,columns)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 2 columns):
 #   Column       Dtype   
---  ------       -----   
 0   user_id      int32   
 1   region_name  category
dtypes: category(1), int32(1)
memory usage: 1.5 GB


In [34]:
# в top_region количество строк с этим регионом у юзера
df = data.groupby(['user_id', 'region_name'])['region_name'].count()
df.name = 'top_region'

display(df.info())
print(df.index)

# удаляем из фрейма города, в которых не был человек
df = df[df != 0].reset_index()
display(df.head(20))
# в скольки регионах бывал пользователь
region_count = df.groupby('user_id', group_keys=False)['region_name'].count()
#оставляем только топовые регионы для каждого пользоватеоля
df = df.sort_values(by='top_region', ascending=False).drop_duplicates(subset='user_id', keep='first', ignore_index=True)
df = df.drop('top_region', axis=1)
#сортируем и добавляем колонку
df = df.sort_values(by='user_id').reset_index(drop=True)
df['region_count'] = region_count

<class 'pandas.core.series.Series'>
MultiIndex: 33640677 entries, (0, 'Алтайский край') to (415316, 'Ярославская область')
Series name: top_region
Non-Null Count     Dtype
--------------     -----
33640677 non-null  int64
dtypes: int64(1)
memory usage: 436.4 MB


None

MultiIndex([(     0,             'Алтайский край'),
            (     0,           'Амурская область'),
            (     0,      'Архангельская область'),
            (     0,       'Астраханская область'),
            (     0,       'Белгородская область'),
            (     0,           'Брянская область'),
            (     0,       'Владимирская область'),
            (     0,      'Волгоградская область'),
            (     0,        'Вологодская область'),
            (     0,        'Воронежская область'),
            ...
            (415316,      'Удмуртская Республика'),
            (415316,        'Ульяновская область'),
            (415316,           'Хабаровский край'),
            (415316, 'Ханты-Мансийский АО — Югра'),
            (415316,        'Челябинская область'),
            (415316,       'Чеченская Республика'),
            (415316,       'Чувашская Республика'),
            (415316,               'Чукотский АО'),
            (415316,          'Ямало-Ненецкий АО

,user_id,region_name,top_region
0,0,Москва,131
1,1,Москва,572
2,1,Московская область,37
3,1,Санкт-Петербург,91
4,2,Республика Коми,356
5,3,Воронежская область,188
6,4,Волгоградская область,23
7,4,Краснодарский край,459
8,4,Республика Башкортостан,97
9,4,Самарская область,11


In [35]:
display(df)
be = BinaryEncoder(cols=['region_name'],return_df=True)
df = be.fit_transform(df)
display(df)

,user_id,region_name,region_count
0,0,Москва,1
1,1,Москва,3
2,2,Республика Коми,1
3,3,Воронежская область,1
4,4,Краснодарский край,5
...,...,...,...
415312,415312,Тамбовская область,1
415313,415313,Саратовская область,1
415314,415314,Новосибирская область,1
415315,415315,Краснодарский край,1


,user_id,region_name_0,region_name_1,region_name_2,region_name_3,region_name_4,region_name_5,region_name_6,region_count
0,0,0,0,0,0,0,0,1,1
1,1,0,0,0,0,0,0,1,3
2,2,0,0,0,0,0,1,0,1
3,3,0,0,0,0,0,1,1,1
4,4,0,0,0,0,1,0,0,5
...,...,...,...,...,...,...,...,...,...
415312,415312,1,0,0,0,1,1,0,1
415313,415313,0,0,1,1,0,0,0,1
415314,415314,0,0,1,0,0,1,1,1
415315,415315,0,0,0,0,1,0,0,1


In [36]:
display(df.info())
print(df['region_count'].max()) #int8
print(df['user_id'].max()) #int32

df[['region_count']] = df[['region_count']].astype('int8')
df[['user_id']] = df[['user_id']].astype('int32')
df[df.columns[~df.columns.isin(['region_count','user_id'])]] = df[df.columns[~df.columns.isin(['region_count','user_id'])]].astype('category')

display(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   user_id        415317 non-null  int64
 1   region_name_0  415317 non-null  int64
 2   region_name_1  415317 non-null  int64
 3   region_name_2  415317 non-null  int64
 4   region_name_3  415317 non-null  int64
 5   region_name_4  415317 non-null  int64
 6   region_name_5  415317 non-null  int64
 7   region_name_6  415317 non-null  int64
 8   region_count   415317 non-null  int64
dtypes: int64(9)
memory usage: 28.5 MB


None

41
415316
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   user_id        415317 non-null  int32   
 1   region_name_0  415317 non-null  category
 2   region_name_1  415317 non-null  category
 3   region_name_2  415317 non-null  category
 4   region_name_3  415317 non-null  category
 5   region_name_4  415317 non-null  category
 6   region_name_5  415317 non-null  category
 7   region_name_6  415317 non-null  category
 8   region_count   415317 non-null  int8    
dtypes: category(7), int32(1), int8(1)
memory usage: 4.8 MB


None

In [38]:
df.to_parquet('..\\full_data\\figma_plan\\region.parquet', index=False)